<h1> Chapter 7 - Medium Tasks</h1>
<i>Going beyond prompt engineering.</i>


## Using OpenAI API
- This notebook uses OpenAI via LangChain (`ChatOpenAI`).
- Set `OPENAI_API_KEY` in your environment before running.
- All tasks use `llm`, which is the OpenAI chat model.

In [ ]:
# OpenAI API setup
import os
from langchain_openai import ChatOpenAI

# Set the API key directly
# os.environ["OPENAI_API_KEY"] = "api-key-here"

print("API key set")

# Fast model
openai_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# Default LLM for the tasks below
llm = openai_llm

API key set


<h2>Medium Tasks: Chains</h2>
<p>Build simple but powerful chains. Keep it step-by-step and observe how each link changes the output.|</p>

<h3>Medium Task 1 - A Single Link: Prompt Template -> LLM</h3>
<p>Goal: Make a minimal prompt template and connect it to the OpenAI chat model.</p>
<ol>
  <li>Create a minimal prompt template.</li>
  <li>Connect the template to the chat model as a simple chain.</li>
  <li>Invoke the chain with a short input and observe the output.</li>
</ol>


In [5]:
# Imports for chains and prompts
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain

# We'll use the OpenAI chat model defined above
llm = openai_llm

In [6]:
from langchain_core.prompts import PromptTemplate

# Minimal prompt template
template = "{input_prompt}"
prompt = PromptTemplate(template=template, input_variables=["input_prompt"])

In [7]:
from langchain_classic.chains import LLMChain

# Single-link chain (prompt -> LLM)
basic_chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipykernel_110123/2165525862.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  basic_chain = LLMChain(llm=llm, prompt=prompt)


In [8]:
# Try a tiny warm-up
result_basic = basic_chain.invoke({"input_prompt": "Hi! My name is Pat. What is 1 + 1?"})
print(result_basic)

{'input_prompt': 'Hi! My name is Pat. What is 1 + 1?', 'text': 'Hi Pat! 1 + 1 equals 2.'}


<h3>Medium Task 2 - Multiple Links: Title -> Character -> Story</h3>
<p>Goal: Break a bigger task into smaller steps (links) and chain them.</p>
<ol>
  <li>Make a title from a short summary.</li>
  <li>Describe the main character using the title + summary.</li>
  <li>Write a short story using title + character + summary.</li>
</ol>
<p>Be playful. Keep each output short so you can see the flow clearly.</p>

In [9]:
# Seed summary used across steps
summary = "a sandwich that dreams of becoming a chef"

<b>Step 1: Title</b> — Create a title from the summary.

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain

title_template = "Create a catchy title for a story about {summary}. Only return the title."
title_prompt = PromptTemplate(template=title_template, input_variables=["summary"])
title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

In [11]:
step1 = title_chain.invoke({"summary": summary})
my_title = step1.get("title", step1.get("text", ""))
print("TITLE:", my_title)

TITLE: "Chef in the Crust: The Dream of a Culinary Sandwich"


<b>Step 2: Character</b> — Describe the protagonist using title + summary.

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain

character_template = "Describe the main character of a story about {summary} titled {title}. Use two sentences."
character_prompt = PromptTemplate(template=character_template, input_variables=["summary", "title"])
character_chain = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [13]:
step2 = character_chain.invoke({"summary": summary, "title": my_title})
my_character = step2.get("character", step2.get("text", ""))
print("CHARACTER:", my_character)

CHARACTER: The main character, Sammy the Sandwich, is a spirited and ambitious hero made of fresh ingredients, with a zest for life and a passion for culinary creativity. Despite being nestled in a bustling deli, Sammy dreams of escaping the confines of his sandwich board to become a renowned chef, crafting gourmet dishes that tantalize taste buds everywhere.


<b>Step 3: Story</b> — Write a short story using title + character + summary.

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain

story_template = (
    "Write a very short story (one paragraph) about {summary} with the title {title}. "
    "The main character is: {character}. "
    "Only return the story."
)
story_prompt = PromptTemplate(template=story_template, input_variables=["summary", "title", "character"])
story_chain = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [15]:
step3 = story_chain.invoke({"summary": summary, "title": my_title, "character": my_character})
my_story = step3.get("story", step3.get("text", ""))
print("STORY:", my_story)

STORY: **Chef in the Crust: The Dream of a Culinary Sandwich**  

Sammy the Sandwich, layered with crisp lettuce, ripe tomatoes, and a hint of zesty mustard, gazed longingly at the bustling kitchen behind the deli counter, where chefs danced with knives and flames, creating culinary masterpieces. Each day, as customers devoured his fellow sandwiches, Sammy's dreams grew bolder; he envisioned himself plating exquisite dishes, transforming simple ingredients into art. With a heart full of ambition and a sprinkle of hope, he plotted his escape, dreaming of a world where he could sprinkle herbs and sauté vegetables, proving that even a humble sandwich could rise to the occasion and become a culinary sensation.


<h3>Reflect</h3>
<ul>
  <li>How did chaining smaller prompts change the quality of the final story?</li>
  <li>What happens if you change the summary to something else?</li>
  <li>Try running the chain again and see if you get different results.</li>
</ul>
<p>If any cell feels slow or costly, use a smaller OpenAI model (e.g., keep <code>gpt-4o-mini</code> or lower temperature) and rerun that section.</p>